Import libraries.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os

Reads the data and display the frist 5 rows.

In [9]:
data = pd.read_csv("/workspaces/CS_Group_09_Salary_prediction/data/Salary_Data.csv")

In [10]:
data.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


After loading the dataset I am planning to ready the dataset for the training.\

Let start by data exploration to filter out missing and incorrect data.

In [11]:
print("Missing values:\n", data.isnull().sum())

Missing values:
 Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
dtype: int64


Lets get the general idea of the dataset by retreiving simple stats by simple calculations of the dataset

In [17]:
print("\nData types:\n", data.dtypes)
print("\nStatistics:\n", data.describe())


Data types:
 Age                    float64
Gender                  object
Education Level         object
Job Title               object
Years of Experience    float64
Salary                 float64
dtype: object

Statistics:
                Age  Years of Experience         Salary
count  6702.000000          6701.000000    6699.000000
mean     33.620859             8.094687  115326.964771
std       7.614633             6.059003   52786.183911
min      21.000000             0.000000     350.000000
25%      28.000000             3.000000   70000.000000
50%      32.000000             7.000000  115000.000000
75%      38.000000            12.000000  160000.000000
max      62.000000            34.000000  250000.000000


Next lets remove the missing raws. Drop any raw with missing values

In [14]:
data_cleaned = data.dropna()

So raws with missing values are romoved now it's the time to take down values that are not natural. Here in this case what I see are Age, Years of Experience, Salary columns. Since Age is not a major fact used in this training I will only consider Years of Experience and Salary columns. All the raws where Years of Experience and salary is 0 or a negative value that should be removed.

In [16]:
data_cleaned = data_cleaned[
    (data_cleaned['Years of Experience'] > 0) & (data_cleaned['Salary'] > 0)
]

Lets checked the removed raws and updated stats of the dataset

In [18]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6578 entries, 0 to 6703
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6578 non-null   float64
 1   Gender               6578 non-null   object 
 2   Education Level      6578 non-null   object 
 3   Job Title            6578 non-null   object 
 4   Years of Experience  6578 non-null   float64
 5   Salary               6578 non-null   float64
dtypes: float64(3), object(3)
memory usage: 359.7+ KB


In [19]:
data_cleaned.describe()

,Age,Years of Experience,Salary
count,6578.000000,6578.000000,6578.000000
mean,33.811493,8.242855,116891.716175
std,7.553069,6.014961,51957.786817
min,22.000000,0.500000,350.000000
25%,28.000000,3.000000,70000.000000
50%,32.000000,7.000000,119000.000000
75%,38.000000,12.000000,160000.000000
max,62.000000,34.000000,250000.000000


Lets see a side by side coparison of the dataset before and after the cleaning

In [27]:
# Original dataset
raw_desc = data.describe().T

# Cleaned dataset
cleaned_desc = data_cleaned.describe().T

# Rename the columns for better understanding
raw_desc.columns = ['Raw_Count', 'Raw_Mean', 'Raw_Std', 'Raw_Min', 'Raw_25%', 'Raw_50%', 'Raw_75%', 'Raw_Max']
cleaned_desc.columns = ['Cleaned_Count', 'Cleaned_Mean', 'Cleaned_Std', 'Cleaned_Min', 'Cleaned_25%', 'Cleaned_50%', 'Cleaned_75%', 'Cleaned_Max']

# Combine side by side to get a clear view 
comparison_df = pd.concat([raw_desc, cleaned_desc], axis=1)

# Display the tables
comparison_df


,Raw_Count,Raw_Mean,Raw_Std,Raw_Min,Raw_25%,Raw_50%,Raw_75%,Raw_Max,Cleaned_Count,Cleaned_Mean,Cleaned_Std,Cleaned_Min,Cleaned_25%,Cleaned_50%,Cleaned_75%,Cleaned_Max
Age,6702.0,33.620859,7.614633,21.0,28.0,32.0,38.0,62.0,6578.0,33.811493,7.553069,22.0,28.0,32.0,38.0,62.0
Years of Experience,6701.0,8.094687,6.059003,0.0,3.0,7.0,12.0,34.0,6578.0,8.242855,6.014961,0.5,3.0,7.0,12.0,34.0
Salary,6699.0,115326.964771,52786.183911,350.0,70000.0,115000.0,160000.0,250000.0,6578.0,116891.716175,51957.786817,350.0,70000.0,119000.0,160000.0,250000.0


Nex we are going to scale the data. It is a very important step befor applying Lasso regression.
Why? 
Lasso regression depends on feature magnitude. Scaling ensures fair weight distribution amomg the dataset.

In [31]:
from sklearn.preprocessing import StandardScaler

X = data_cleaned[['Years of Experience']]
y = data_cleaned['Salary']

# Scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Before traing the dataset it is split in 20% and 80%, 80% training the dataset and remaining 20% to do the testing. We do this because we need to train in a idfferent part adn test in another part. Becasue if we do the train and test in the same part of the dataset it will always give very accurate answers which is missleading because the model might have already memorized the answers which leads to overfiting.

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)